# Main

In [ ]:
import hashlib
import timeit
import random
import math

In [ ]:
class RSA:
    def __init__(self, key_size):
        self.prime1 = self.generate_prime(key_size)
        self.prime2 = self.generate_prime(key_size)
        self.modulus = self.prime1 * self.prime2
        self.totient = self.calculate_lcm(self.prime1 - 1, self.prime2 - 1)
        self.public_key = self.find_public_key()
        self.private_key = self.find_private_key()

    def calculate_lcm(self, num1, num2):
        return abs(num1 * num2) // math.gcd(num1, num2)

    def miller_rabin(self, candidate, rounds=5):
        if candidate in (2, 3):
            return True
        if candidate <= 1 or candidate % 2 == 0:
            return False
        r, s = 0, candidate - 1
        while s % 2 == 0:
            r += 1
            s //= 2
        for _ in range(rounds):
            a = random.randrange(2, candidate - 1)
            x = pow(a, s, candidate)
            if x in (1, candidate - 1):
                continue
            for _ in range(r - 1):
                x = pow(x, 2, candidate)
                if x == candidate - 1:
                    break
            else:
                return False
        return True

    def generate_prime(self, key_size):
        while True:
            num = random.getrandbits(key_size)
            if self.miller_rabin(num):
                return num

    def find_public_key(self):
        e = 2
        while self.compute_gcd(e, self.totient) != 1:
            e += 1
        return e

    def find_private_key(self):
        d = 2
        while (d * self.public_key) % self.totient != 1:
            d += 1
        return d

    def compute_gcd(self, num1, num2):
        while num2:
            num1, num2 = num2, num1 % num2
        return num1

    def encrypt_message(self, plaintext):
        return [pow(ord(char), self.public_key, self.modulus) for char in plaintext]

    def decrypt_message(self, ciphertext):
        return ''.join(chr(pow(char, self.private_key, self.modulus)) for char in ciphertext)

    def generate_signature(self, message):
        hash_digest = int(hashlib.sha1(message.encode()).hexdigest(), 16)
        hash_digest &= (1 << key_size) - 1
        return pow(hash_digest, self.private_key, self.modulus)

    def verify_signature(self, message, signature):
        hash_digest = int(hashlib.sha1(message.encode()).hexdigest(), 16)
        hash_digest &= (1 << key_size) - 1
        decrypted_hash = pow(signature, self.public_key, self.modulus)
        return hash_digest == decrypted_hash


In [ ]:
class Sender:
    def __init__(self, crypto_system):
        self.crypto_system = crypto_system

    def send(self, message):
        encrypted_message = self.crypto_system.encrypt_message(message)
        signature = self.crypto_system.generate_signature(message)
        return encrypted_message, signature


In [ ]:
class Receiver:
    def __init__(self, crypto_system):
        self.crypto_system = crypto_system

    def receive(self, encrypted_message, signature):
        decrypted_message = self.crypto_system.decrypt_message(encrypted_message)
        if self.crypto_system.verify_signature(decrypted_message, signature):
            print("Signature is valid.")
        else:
            print("Signature is invalid.")
        return decrypted_message

In [ ]:
def compute_hash(message):
    return hashlib.sha1(message.encode()).hexdigest()

start_time = timeit.default_timer()

key_size = 256
rsa = RSA(key_size)

alice = Sender(rsa)
bob = Receiver(rsa)

message = "Univer zadobav ... "
print("Original message: ", message)

In [ ]:
encrypted_message, signature = alice.send(message)
print("Encrypted message: ", encrypted_message)
print("Signature: ", signature)

In [ ]:
decrypted_message = bob.receive(encrypted_message, signature)
print("Decrypted message: ", decrypted_message)

In [ ]:
message_hash = compute_hash(message)
print("Hash of the message: ", message_hash)

In [ ]:
elapsed_time = timeit.default_timer() - start_time
print("Execution time: ", elapsed_time)